In [1]:
import QSTK.qstkutil.DataAccess as da
import QSTK.qstkutil.qsdateutil as du

/home/rahul/app_packages/anaconda3/envs/qstkpython2env/lib/python2.7/site-packages/QSTK/qstkutil/qsdateutil.py:36: FutureWarning: TimeSeries is deprecated. Please use Series
  return pd.TimeSeries(index=dates, data=dates)


In [2]:
import datetime as dt
import numpy as np
import pandas as pd

In [3]:
pd.set_option('max_rows', 20)

In [4]:
AVAILABLE_PRICE_TYPES = ('open', 'high', 'low', 'close', 'volume', 'actual_close')


def get_price_dict(dt_start, dt_end, symbols, ls_price_type):
    assert all(price_type in AVAILABLE_PRICE_TYPES for price_type in ls_price_type)
    days = du.getNYSEdays(dt_start, dt_end, dt.timedelta(hours=16))
    c_dataobj = da.DataAccess('Yahoo')
    data_list = c_dataobj.get_data(days, symbols, ls_price_type)
    return dict(zip(ls_price_type, data_list))


def get_price_df(dt_start, dt_end, symbols, price_type='close'):
    assert price_type in AVAILABLE_PRICE_TYPES
    d_data = get_price_dict(dt_start, dt_end, symbols, [price_type])    
    return d_data[price_type]

# Example usage
# get_price_df(dt.date(2012, 1, 1), dt.date(2012, 1, 15), ["AAPL", "MSFT"], 'actual_close')

In [5]:
def create_orders_df(filepath):
    orders = pd.read_csv(filepath, header=-1)
    # delete last column
    orders = orders.ix[:, range(len(orders.columns) -1)]
    orders.columns = ['year', 'month', 'day', 'symbol', 'side', 'quantity']
    orders['date'] = pd.to_datetime(orders[['year', 'month', 'day']])
    return orders[['date', 'symbol', 'side', 'quantity']].copy()

In [6]:
def compute_fund_valuation(orderfile, initial_cash):
    df_orders_all = create_orders_df(orderfile).sort_values(by='date')
    df_orders_all['signed_qty'] = df_orders_all.apply(lambda row: row['quantity'] * (1 if row['side'] == 'Buy' else -1), axis=1)
    dt_start = df_orders_all.iloc[0]['date'].date()
    dt_end = df_orders_all.iloc[-1]['date'].date()
    symbols = df_orders_all['symbol'].unique().tolist()
    df_close_all = get_price_df(dt_start, dt_end, symbols, price_type='close')
    df_close_all.index = [x.date() for x in df_close_all.index]
    df_value_all = pd.DataFrame(index=df_close_all.index)
    df_cash = pd.Series(len(df_close_all.index)*[initial_cash], index=df_close_all.index)
    for symbol in symbols:
        df_orders = df_orders_all[df_orders_all['symbol'] == symbol]
        df_orders = df_orders['signed_qty'].groupby(df_orders['date']).sum()
        df_orders = df_orders.reindex(df_close_all.index).fillna(0)
        df_close = df_close_all[symbol]
        df_cash_outflow = (df_close * df_orders).cumsum()               
        df_holding = df_orders.cumsum()        
        df_value = df_close * df_holding
        df_cash = df_cash - df_cash_outflow
        df_value_all[symbol] = df_value
    
    df_value_all['cash'] = df_cash
    df_value_all['total'] = df_value_all.apply(sum, axis=1)
    return df_value_all

In [7]:
!cat data/orders-short.csv

2011,1,05,AAPL,Buy,1500,
2011,1,20,AAPL,Sell,1500,


In [8]:
df_daily_fund_valuation = compute_fund_valuation('data/orders-short.csv', 1000000)

In [9]:
df_daily_fund_valuation

,AAPL,cash,total
2011-01-05,498855.0,501145.0,1000000.0
2011-01-06,498450.0,501145.0,999595.0
2011-01-07,502020.0,501145.0,1003165.0
2011-01-10,511485.0,501145.0,1012630.0
2011-01-11,510270.0,501145.0,1011415.0
2011-01-12,514425.0,501145.0,1015570.0
2011-01-13,516300.0,501145.0,1017445.0
2011-01-14,520485.0,501145.0,1021630.0
2011-01-18,508785.0,501145.0,1009930.0
2011-01-19,506085.0,501145.0,1007230.0


In [10]:
def generate_performance_metrics(daily_ret_series):
    avg_ret = np.nanmean(daily_ret_series)
    std_ret = np.nanstd(daily_ret_series)
    sharpe_ratio = np.sqrt(252) * avg_ret / std_ret
    return avg_ret, std_ret, sharpe_ratio    

In [11]:
def generate_fund_performance_metrics(orderfile, initial_cash):
    df_daily_fund_valuation = compute_fund_valuation(orderfile, initial_cash)
    df_fund_daily_ret = df_daily_fund_valuation['total'] / df_daily_fund_valuation['total'].shift(1) - 1.0
    dt_start, dt_end = df_daily_fund_valuation.index[0], df_daily_fund_valuation.index[-1]
    df_spx_close = get_price_df(dt_start, dt_end, ['$SPX'], price_type='close')
    df_spx_close = df_spx_close.fillna(method='ffill').fillna(method='bfill')
    df_spx_ret = df_spx_close / df_spx_close.shift(1) - 1.0
    fund_avg_ret, fund_std_ret, fund_sharpe_ratio = generate_performance_metrics(df_fund_daily_ret)
    spx_avg_ret, spx_std_ret, spx_sharpe_ratio = generate_performance_metrics(df_spx_ret)
    print("Details of the Performance of the portfolio :")
    print("Data Range : " + str(dt_start) + " to " + str(dt_end))
    print("\n")
    print("Sharpe Ratio of Fund : " + str(fund_sharpe_ratio))
    print("Sharpe Ratio of $SPX : " + str(spx_sharpe_ratio))
    print("\n")
    print("Standard Deviation of Fund: " + str(fund_std_ret))
    print("Standard Deviation of $SPX: " + str(spx_std_ret))
    print("\n")
    print("Average Daily Return of Fund : " + str(fund_avg_ret))
    print("Average Daily Return of $SPX : " + str(spx_avg_ret))    
    return df_daily_fund_valuation

In [12]:
df = generate_fund_performance_metrics('data/orders-short.csv', 1000000)

Details of the Performance of the portfolio :
Data Range : 2011-01-05 to 2011-01-20


Sharpe Ratio of Fund : -0.471124970325
Sharpe Ratio of $SPX : 0.929887436185


Standard Deviation of Fund: 0.00601586846767
Standard Deviation of $SPX: 0.00516969330861


Average Daily Return of Fund : -0.000178539446839
Average Daily Return of $SPX : 0.000302827205547


In [13]:
!cat data/orders2.csv

2011,1,14,AAPL,Buy,1500,
2011,1,19,AAPL,Sell,1500,
2011,1,19,IBM,Buy,4000,
2011,1,31,GOOG,Buy,1000,
2011,2,4,XOM,Sell,4000,
2011,2,11,XOM,Buy,4000,
2011,3,2,GOOG,Sell,1000,
2011,3,2,IBM,Sell,2200,
2011,6,2,IBM,Sell,3300,
2011,5,23,IBM,Buy,1500,
2011,6,10,AAPL,Buy,1200,
2011,8,9,GOOG,Buy,55,
2011,8,11,GOOG,Sell,55,
2011,12,14,AAPL,Sell,1200,


In [14]:
df = generate_fund_performance_metrics('data/orders2.csv', 1000000)

Details of the Performance of the portfolio :
Data Range : 2011-01-14 to 2011-12-14


Sharpe Ratio of Fund : 0.79069560435
Sharpe Ratio of $SPX : -0.177587826049


Standard Deviation of Fund: 0.00709561228768
Standard Deviation of $SPX: 0.0150238604299


Average Daily Return of Fund : 0.000353426354584
Average Daily Return of $SPX : -0.000168071648902


In [15]:
df = generate_fund_performance_metrics('data/week5/orders.csv', 1000000)

Details of the Performance of the portfolio :
Data Range : 2011-01-10 to 2011-12-20


Sharpe Ratio of Fund : 1.21795959093
Sharpe Ratio of $SPX : 0.0183774676298


Standard Deviation of Fund: 0.00719005204538
Standard Deviation of $SPX: 0.0149402549625


Average Daily Return of Fund : 0.000551651296638
Average Daily Return of $SPX : 1.7295909534e-05


In [16]:
df.ix[dt.date(year=2011, month=12, day=6)]

AAPL      467136.0
IBM            0.0
GOOG           0.0
XOM            0.0
cash      660748.0
total    1127884.0
Name: 2011-12-06, dtype: float64

In [17]:
df.ix[dt.date(year=2011, month=11, day=9)]

AAPL      472308.0
IBM            0.0
GOOG           0.0
XOM            0.0
cash      660748.0
total    1133056.0
Name: 2011-11-09, dtype: float64

In [18]:
df.ix[dt.date(year=2011, month=9, day=26)]

AAPL      481740.0
IBM            0.0
GOOG           0.0
XOM            0.0
cash      660748.0
total    1142488.0
Name: 2011-09-26, dtype: float64

In [19]:
df.ix[dt.date(year=2011, month=7, day=21)]

AAPL      462756.0
IBM            0.0
GOOG           0.0
XOM            0.0
cash      660748.0
total    1123504.0
Name: 2011-07-21, dtype: float64

In [20]:
df.ix[dt.date(year=2011, month=3, day=28)]

AAPL           0.0
IBM       283248.0
GOOG           0.0
XOM            0.0
cash      769519.0
total    1052767.0
Name: 2011-03-28, dtype: float64

In [21]:
df2 = generate_fund_performance_metrics('data/week5/orders2.csv', 1000000)

Details of the Performance of the portfolio :
Data Range : 2011-01-14 to 2011-12-14


Sharpe Ratio of Fund : 0.79069560435
Sharpe Ratio of $SPX : -0.177587826049


Standard Deviation of Fund: 0.00709561228768
Standard Deviation of $SPX: 0.0150238604299


Average Daily Return of Fund : 0.000353426354584
Average Daily Return of $SPX : -0.000168071648902


In [22]:
df2.ix[dt.date(year=2011, month=12, day=6)]

AAPL      467136.0
IBM            0.0
GOOG           0.0
XOM            0.0
cash      624480.6
total    1091616.6
Name: 2011-12-06, dtype: float64

In [23]:
df2.ix[dt.date(year=2011, month=11, day=9)]

AAPL      472308.0
IBM            0.0
GOOG           0.0
XOM            0.0
cash      624480.6
total    1096788.6
Name: 2011-11-09, dtype: float64

In [24]:
df2.ix[dt.date(year=2011, month=9, day=26)]

AAPL      481740.0
IBM            0.0
GOOG           0.0
XOM            0.0
cash      624480.6
total    1106220.6
Name: 2011-09-26, dtype: float64

In [25]:
df2.ix[dt.date(year=2011, month=7, day=21)]

AAPL      462756.0
IBM            0.0
GOOG           0.0
XOM            0.0
cash      625101.0
total    1087857.0
Name: 2011-07-21, dtype: float64

In [26]:
df2.ix[dt.date(year=2011, month=3, day=28)]

AAPL           0.0
IBM       283248.0
GOOG           0.0
XOM            0.0
cash      724866.0
total    1008114.0
Name: 2011-03-28, dtype: float64